Econometría Aplicada. Lección 5
===============================

**Author:** Marcos Bujosa



<div class="ABSTRACT" id="org3426b19">
<p>
En esta lección veremos algunas herramientas estadísticas.
</p>

</div>



$
\newcommand{\lag}{\mathsf{B}}
\newcommand{\Sec}[1]{\boldsymbol{#1}}
\newcommand{\Pol}[1]{\boldsymbol{#1}}
$



#### Carga de algunas librerías de R



Primero cargamos la librería `tfarima` (Repositorio Cran:
[https://cran.r-project.org/web/packages/tfarima/index.html](https://cran.r-project.org/web/packages/tfarima/index.html);
repositorio GitHub: [https://github.com/gallegoj/tfarima](https://github.com/gallegoj/tfarima))



In [1]:
library(tfarima)      # librería de José Luis Gallego para Time Series
#library(latticeExtra) # para gráficos con doble eje vertical (doubleYScale)
library(readr)        # para leer ficheros CSV
library(ggplot2)      # para el scatterplot (alternaticamente library(tidyverse))
library(ggfortify)    # para pintar series temporales
library(jtools)       # para representación resultados estimación
library(zoo)          # para generar objetos ts (time series)

y además fijamos los parámetros por defecto para las figuras en `png`
del notebook



In [1]:
# fijamos el tamaño de las figuras que se generan en el notebook
options(repr.plot.width = 12, repr.plot.height = 4, repr.plot.res = 200)

## Series temporales



Corresponden a mediciones de un mismo objeto a lo largo del tiempo. El
índice indica el instante de cada medición.  *El orden cronológico de
los datos puede ser fundamental* para explicar cada uno de ellos.

-   El motivo es que con frecuencia la medición en un instante de tiempo
    está relacionada con otras mediciones próximas en el tiempo
    (*correlación serial*).

-   Si es así, no podremos asumir que las variables aleatorias del
    proceso estocástico subyacente, $\boldsymbol{X}=(X_t\mid t\in\mathbb{Z})$, sean
    independientes entre sí.

Esto tiene importantes implicaciones en las técnicas de análisis a
aplicar y el tipo de modelos a utilizar.



##### Población en Australia



In [1]:
PoblacionAustralia_ts = as.ts( read.zoo('datos/PoblacionAustralia.csv', 
                                        header=TRUE,
                                        index.column = 1, 
                                        sep=",", 
                                        FUN = as.yearmon))
autoplot(PoblacionAustralia_ts)

![img](./img/lecc05/PoblacionAustralia.png)



##### PIB UEM



In [1]:
PIB_UEM_ts = as.ts( read.csv.zoo("datos/PIB_UEM.csv", 
                                 FUN = as.yearqtr, 
                                 format = "%YQ%q", 
                                 strip.white = TRUE))
autoplot(PIB_UEM_ts)

![img](./img/lecc05/PIB_UEM.png)



##### Rendimiento porcentual diario del IBEX 35



In [1]:
IBEX35_ts = as.ts( read.csv.zoo("datos/IBEX35.csv", 
                                strip.white = TRUE))
autoplot(IBEX35_ts)

![img](./img/lecc05/IBEX35.png)



##### Producción de cemento



In [1]:
ProduccionCemento_ts = as.ts( read.csv.zoo("datos/ProduccionCemento.csv",
                                           FUN = as.yearmon, 
                                           format = "%YM%m",
                                           strip.white = TRUE))
autoplot(ProduccionCemento_ts)

![img](./img/lecc05/ProduccionCemento.png)



## Correlación serial vs muestreo aleatorio simple



-   Con datos de sección cruzada se asume muestreo aleatorio simple
    -   i.e., los datos son realizaciones de variables aleatorias i.i.d.

-   Con series temporales dicha asunción resulta generalmente errónea
    
    -   con frecuencia el valor esperado parece cambiar con $t$
    -   con frecuencia hay dependencia temporal correlación serial
    
    **Ejemplo**: dada la evolución de los datos, no parece aceptable
    asumir que $ProdCemento_{1960M01}$ tiene la misma distribución que
    $ProdCemento_{2000M04}$

Veamos por qué esto genera dificultades&#x2026;



Consideremos el proceso estocástico $$\boldsymbol{X}=(X_t \mid
t=0,\pm1,\pm2,\ldots).$$ Caracterizar su distribución conjunta (todos
los momentos) es demasiado ambicioso.



Tentativamente vamos a fijarnos únicamente en los dos primeros momentos:

$$E(X_t)={\color{blue}{ \mu_t}}\qquad\text{ y } \qquad
Cov(X_t,X_k)=E\big[(X_t-\mu_t)(X_k-\mu_k)\big]={\color{blue}{\lambda_{t,k}}}$$
(si $\;k=t\;$ entonces $\;\lambda_{t,t}=Var(X_t)=\sigma^2_t$).

Si las $X_t$ fueran gaussianas, conocer estos *parámetros* bastaría
para caracterizar la distribución conjunta. Pero aún así&#x2026;



-   necesitaríamos una muestra suficiente de cada $X_t$ para estimar los parámetros 
    -   y en una serie temporal tenemos una única realización para cada $X_t$

-   y además hay más parámetros que variables aleatorias



### Casos que simplifican el escenario



-   Cuando el proceso es [débilmente estacionario (Lecc01)](./Lecc01.slides.html#/3/1) se reduce
    drásticamente el número de parámetros.
    
    \begin{eqnarray}
    E(X_t)  & = \mu \\
    Cov(X_t,X_{t-k}) & = \gamma_k
    \end{eqnarray}

-   Si además pudiéramos asumir que el proceso es i.i.d. podríamos
    interpretar la serie temporal como una realización de un muestreo
    aleatorio simple (lo que habilita la inferencia estadística).



El desafío para el analista es (y nótese el abuso de lenguaje)

-   **primero:** transformar los datos para lograr que sean "***estacionarios***"
    -   (Lo vimos en la lección 1))
-   **después:** transformar los datos estacionarios en una secuencia de
    "**datos *i.i.d***"
    
    -   (es lo que nos falta por ver)



## Función de autocovarianzas y función de autocorrelación



-   La secuencia $(\gamma_k)$ con $k\in\mathbb{Z}$ se denomina
    *función de autocovarianzas*

-   La secuencia $\{\rho_k\}$ con $k\in\mathbb{Z}$, donde
    
    $$\rho_k=\frac{Cov(X_t,X_{t-k})}{\sqrt{Var(X_t)Var(X_{t-k})}}=\frac{\gamma_k}{\gamma_0} $$
    
    se denomina *función de autocorrelación* (ACF).



Debido a la estacionariedad, la correlación entre $X_t$ y $X_{t+k}$ no
depende de $t$; tan solo depende de la distancia temporal $k$ entre
ambas variables.



$$\boldsymbol{\phi}(\mathsf{B}) : \phi  $$

